In [23]:
# T bounds 324-524 K
# P bounds 0.1-36 atm
# Steady-state operation with ideal mixing, ideal phase equilibrium, and ideal gas behaviour.
import numpy as np
from scipy.optimize import fsolve
# stream and process unit tracker class
class Streams():
    _instances = []
    def __init__(self,name):
        self.__class__._instances.append(self)
        self.name = name
    @classmethod
    def get_all(cls):
        return cls._instances

    @classmethod
    def clear_registry(cls):
        cls._instances = []


class Stream(Streams):
    def __init__(self, name, T, P, F,z=[0,0,0,0]):
        # 1 Ethylene Oxide
        # 2 Water
        # 3 Ethylene Glycol
        # 4 Diethylene Glycol
        super().__init__(name)
        self.T = T
        self.P = P
        self.F = F
        self.z = np.array(z)
        self.x = z
        self.y = np.zeros(4)
    def __str__(self):
        return f"Stream {self.name}: T={self.T} K, P={self.P} atm, F={self.F} kmol/h, z={self.z}, x={self.x}, y={self.y}"

    def saturation_pressure(self):
        sat_data = np.array([
            [10.884,3152,7.667],
            [11.680,3828,-45.412],
            [11.963,4764,-72.275],
            [11.256,4655,103.551],
            ])
        psats= np.exp(sat_data[:,0] - sat_data[:,1]/(sat_data[:,2]+self.T))
        return psats
        
    def liquid_vapor_molar_fractions(self):
        #kmol/m3
        lmdp = np.array([
            [1.655, 0.126300,-0.0002286],
            [75.22, -0.065050, 0],
            [23.66, -0.018040, 0],
            [13.59, -0.009986, 0],
            ])
        
        liquid_molar_densities = lmdp[:,0] + lmdp[:,1]*self.T + lmdp[:,2]*self.T**2
        return liquid_molar_densities

    def H(self):
        hl_data = np.array([
            [-143.8, 0.2037],
            [-320, 0.1068],
            [-519, 0.1924],
            [-723, 0.3043],
            ])

        hv_data = np.array([
            [-74.6, 0.0697],
            [-252.3, 0.0348],
            [-424.3, 0.1030],
            [-603.4, 0.1823],
            ])
        
        hls = hl_data[:,0] + hl_data[:,1]*self.T
        hvs = hv_data[:,0] + hv_data[:,1]*self.T
        
        hl = sum(hls*self.x)*1000
        hv = sum(hvs*self.y)
        return self.F*hl

class ProcessUnits():
    _instances = []
    def __init__(self, name):
        self.__class__._instances.append(self)
        self.name = name

    @classmethod
    def get_all(cls):
        return cls._instances.copy()

    @classmethod
    def clear_registry(cls):
        cls._instances = []
        
class Heater(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T > Tout:
            raise ValueError("Heater inlet temperature must be less than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = self.outlet.H() - inlet_stream.H()

class Cooler(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T < Tout:
            raise ValueError("Cooler inlet temperature must be greater than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = inlet_stream.H() - self.outlet.H()

class Mixer(ProcessUnits):
    def __init__(self, name, inlet_stream1, inlet_stream2, inlet_stream3=None, inlet_stream4=None):
        super().__init__(name)
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.inlet_stream3 = inlet_stream3
        self.inlet_stream4 = inlet_stream4
        P = min(inlet_stream1.P, inlet_stream2.P, (inlet_stream3.P if inlet_stream3 else float('inf')), (inlet_stream4.P if inlet_stream4 else float('inf')))
        F = inlet_stream1.F + inlet_stream2.F + (inlet_stream3.F if inlet_stream3 else 0) + (inlet_stream4.F if inlet_stream4 else 0)
        T = (inlet_stream1.F*inlet_stream1.T + inlet_stream2.F*inlet_stream2.T + (inlet_stream3.F*inlet_stream3.T if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.T if inlet_stream4 else 0)) / F
        z = (inlet_stream1.F*inlet_stream1.z + inlet_stream2.F*inlet_stream2.z + (inlet_stream3.F*inlet_stream3.z if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.z if inlet_stream4 else 0)) / F
        self.outlet = Stream(name+"_outlet", T, P, F, z)

class CSTReactor(ProcessUnits):
    def __init__(self, name, inlet_stream,volume=0):
        super().__init__(name)
        self.inlet_stream = inlet_stream
        self.volume = volume
    def reaction_rates(self, T):
        """Calculate reaction rates r1 and r2 at given temperature T."""
        k1 = 13706.91 * np.exp(-8220 / T)
        k2 = 96341.59 * np.exp(-8700 / T)
        return k1, k2
    

In [ ]:
# T Kelvin, P atm, n kmol/h, z1 mol fraction of Ethylene oxide, z2 mol fraction of water
# Feed1 = Stream('Feed1', 298,2.4,26.32,z=[0,1,0,0])
# Feed2 = Stream('Feed2', 298,2.4,27.62,z=[1,0,0,0])
# D1 = Stream('D1', 360.5,2.4,522.37,z=[0.00164634263,0.99787506939,0.00047858798,0])
# Mixer1 = Mixer('Mixer1', Feed1, Feed2, D1)
# Heater1 = Heater('Heater1', Mixer1.outlet, 355)
# print(Heater1.outlet)
# print(Heater1.outlet.liquid_vapor_molar_fractions())
# print(Heater1.outlet.F*Heater1.outlet.z)
# v = np.sum(Heater1.outlet.F*Heater1.outlet.z/Heater1.outlet.liquid_vapor_molar_fractions())
# print(f"Volumetric flow: {v} m3/hr")
# print(f"Molar flow: {Heater1.outlet.F} kmol/hr")
# print(Heater1.outlet.F*Heater1.outlet.z/v)
CSTRInlet = Stream('CSTRInlet', 355, 2.4, 576.317942978094, z=[0.0494255625707587,0.950134575786539,0.000439861459002414,1.83699843408081E-10])
v = np.sum(CSTRInlet.F*CSTRInlet.z/CSTRInlet.liquid_vapor_molar_fractions())
print(f"Volumetric flow: {v} m3/hr")
print(f"Molar flow: {CSTRInlet.F} kmol/hr")
print(CSTRInlet.F*CSTRInlet.z/v)

Volumetric flow: 12.130295903042116 m3/hr
Molar flow: 576.317942978094 kmol/hr
[2.34823938e+00 4.51414878e+01 2.08980929e-02 8.72769442e-09]


In [115]:
#heat of reaction calculation
#R1
dhr1 = []
dhr2 = []
for T in range(298, 525):
    reactants = Stream("reactants", T,1,2,[0.5,0.5,0,0])
    products = Stream("products", T,1,1,[0,0,1,0])
    heat_of_reaction = products.H() - reactants.H()
    dhr1.append(heat_of_reaction)
    #R2
    reactants2 = Stream("reactants2", T,1,2,[0.5,0,0.5,0])
    products2 = Stream("products2", T,1,1,[0,0,0,1])
    heat_of_reaction2 = products2.H() - reactants2.H()
    dhr2.append(heat_of_reaction2)
dhr1 = np.array(dhr1)
dhr2 = np.array(dhr2)
m,b = np.polyfit(range(298, 525), dhr1, 1)
print(f"R1: dH = {m}*T + {b}")
m,b = np.polyfit(range(298, 525), dhr2, 1)
print(f"R2: dH = {m}*T + {b}")

R1: dH = -118.1*T + -55199.99999999998
R2: dH = -91.79999999999991*T + -60200.00000000002


In [55]:
flow_data_array = np.array([
    [0.00, 355.0, 547.58, 28.48, 0.25, 0.00],
    [3.75, 359.3, 545.49, 26.38, 2.33, 0.02],
    [7.50, 365.0, 542.85, 23.69, 4.93, 0.06],
    [11.25, 372.6, 539.35, 20.09, 8.32, 0.17],
    [15.00, 382.7, 534.70, 15.22, 12.75, 0.39],
    [18.75, 394.4, 529.35, 9.50, 17.72, 0.76],
    [22.50, 403.8, 525.05, 4.81, 21.65, 1.14],
    [26.25, 409.3, 522.59, 2.11, 23.86, 1.38],
    [30.00, 411.8, 521.46, 0.87, 24.87, 1.50]
])
#difference between stepsp
print("DV, DT, DW, DEO, DEG, DDEG")
print(flow_data_array[1,:]-flow_data_array[0,:])
print(flow_data_array[2,:]-flow_data_array[1,:])
print(flow_data_array[3,:]-flow_data_array[2,:])

DV, DT, DW, DEO, DEG, DDEG
[ 3.75  4.3  -2.09 -2.1   2.08  0.02]
[ 3.75  5.7  -2.64 -2.69  2.6   0.04]
[ 3.75  7.6  -3.5  -3.6   3.39  0.11]


In [43]:
import numpy as np
from scipy.optimize import fsolve

# Reactor parameters (units: kmol/m³, kJ/kmol, K)
V =3.75 # m³
Q_flow = 12.130295903042116 #m³/hr
F = 576.317942978094 # kmol/hr
C_A0 = 2.34823938e+00 # kmol/m³
C_B0 = 4.51414878e+01 # kmol/m³
C_C0 = 2.08980929e-02 # kmol/m³
C_D0 = 8.72769442e-09 # kmol/m³
T0 = 355.0 # K

# Stoichiometric coefficients
nu = np.array([[-1, -1, 1, 0], [-1, 0, -1, 1]])

hl_data = np.array([
    [-143.8, 0.2037],  
    [-320.0, 0.1068],  
    [-519.0, 0.1924],  
    [-723.0, 0.3043],  
])

def H(T):
    return (hl_data[:, 0] + hl_data[:, 1] * T)*1000

def rate(C, T):
    C_A, C_B, C_C, C_D = C
    k1 = 13706.91 * np.exp(-8220 / T)
    r1 = k1 * C_A * C_B
    k2 = 96341.59 * np.exp(-8700 / T)
    r2 = k2 * C_A * C_C
    return r1, r2

h_A0, h_B0, h_C0, h_D0 = H(T0)

def equations(vars):
    C_A, C_B, C_C, C_D, T = vars
    
    r1, r2 = rate([C_A, C_B, C_C, C_D], T)
    delta_H_rxn1 = (-0.1181 * T - 55.2)*1000 # kJ/kmol
    delta_H_rxn2 = (-0.0918 * T - 60.2)*1000
    print((r1*V/Q_flow+r2*V/Q_flow))
    resid_A = C_A0 - C_A + (V/Q_flow) * (nu[0,0]*r1 + nu[1,0]*r2) # kmol/m³ - (kmol/m³ + m3/m3/hr * kmol/m³/hr) = kmol/m³
    resid_B = C_B0 - C_B + (V/Q_flow) * (nu[0,1]*r1 + nu[1,1]*r2)
    resid_C = C_C0 - C_C + (V/Q_flow) * (nu[0,2]*r1 + nu[1,2]*r2)
    resid_D = C_D0 - C_D + (V/Q_flow) * (nu[0,3]*r1 + nu[1,3]*r2) 
    
    h_A, h_B, h_C, h_D = H(T)
    hin = (C_A0 * h_A0 + C_B0 * h_B0 + C_C0 * h_C0 + C_D0 * h_D0)*Q_flow # (kmol/m³ * kJ/kmol) = kJ/hr
    hout = (C_A * h_A + C_B * h_B + C_C * h_C + C_D * h_D) * Q_flow # (kmol/m³ * kJ/kmol) = kJ/hr
    print(hin, hout)
    delta_h = hin - hout # kJ/hr
    print(delta_h)
    Q_rxn = (-delta_H_rxn1 * r1 - delta_H_rxn2 * r2) * V # (kj/kmol * kmol/m3/hr) * m3 = kj/hr
    print(Q_rxn)
    resid_T = ((delta_h + Q_rxn) - (np.sum(hl_data[:, 0] *[C_A, C_B, C_C, C_D]))*Q_flow) / (np.sum(hl_data[:, 1] * [C_A, C_B, C_C, C_D])*Q_flow) # ((kj/hr) - (kj/kmol * kmol/m3)* m3/hr) / (kj/kmolK * kmol/m3 * m3/hr) = K 
    print(resid_T)
    return [resid_A, resid_B, resid_C, resid_D, resid_T]

solution = equations([2.17,44.85,0.02,0.0,359.27])
# Adjusted initial guess
guess = [C_A0, C_B0, C_C0, C_D0, T0]
# solution = fsolve(equations, guess)


# Print results
C_A_sol, C_B_sol, C_C_sol, C_D_sol, T_sol = solution
print("Adiabatic CSTR Results:")
print(f"C_A: {C_A_sol:.3f} kmol/m³")
print(f"C_B: {C_B_sol:.3f} kmol/m³")
print(f"C_C: {C_C_sol:.3f} kmol/m³")
print(f"C_D: {C_D_sol:.3f} kmol/m³")
print(f"Temperature: {T_sol:.2f} K")

4.776827343828081e-05
-156615073.76050922 -155186995.6106722
-1428078.149837017
56.56880285863053
-19681.42578930501
Adiabatic CSTR Results:
C_A: 0.178 kmol/m³
C_B: 0.291 kmol/m³
C_C: 0.001 kmol/m³
C_D: 0.000 kmol/m³
Temperature: -19681.43 K


In [76]:
import numpy as np
from scipy.optimize import least_squares

# Reactor parameters (units: kmol/m³, kJ/kmol, K)
V = 3.75  # m³
Q_flow = 12.129827011009338  # m³/hr (volumetric flow rate)
F = 576.31  # kmol/hr (molar flow rate)
C_A0 = 2.34793126  # kmol/m³
C_B0 = 45.1432654  # kmol/m³
C_C0 = 0.0206103519  # kmol/m³
C_D0 = 0.0  # kmol/m³
T0 = 355.0  # K

# Stoichiometric coefficients
nu = np.array([[-1, -1, 1, 0], [-1, 0, -1, 1]])

# Enthalpy data [H0, slope] for A, B, C, D
hl_data = np.array([
    [-143.8, 0.2037],  # A
    [-320.0, 0.1068],  # B
    [-519.0, 0.1924],  # C
    [-723.0, 0.3043],  # D
])

def H(T):
    """Calculate enthalpy (kJ/kmol) for each species at temperature T."""
    return hl_data[:, 0] + hl_data[:, 1] * T

def rate(C, T):
    """Calculate reaction rates for two reactions."""
    C_A, C_B, C_C, C_D = C
    k1 = 13706.91 * np.exp(-8220 / T)
    r1 = k1 * C_A * C_B
    k2 = 96341.59 * np.exp(-8700 / T)
    r2 = k2 * C_A * C_C
    return r1, r2

def equations(vars):
    """System of equations for the adiabatic CSTR (Temperature-based)."""
    C_A, C_B, C_C, C_D, T = vars
    r1, r2 = rate([C_A, C_B, C_C, C_D], T)
    delta_H_rxn1 = (-0.1181 * T - 55.2)  # kJ/kmol
    delta_H_rxn2 = (-0.0918 * T - 60.2)  # kJ/kmol

    # Material Balances (m³ basis)
    resid_A = C_A0 - C_A + (V / Q_flow) * (nu[0, 0] * r1 + nu[1, 0] * r2)
    resid_B = C_B0 - C_B + (V / Q_flow) * (nu[0, 1] * r1 + nu[1, 1] * r2)
    resid_C = C_C0 - C_C + (V / Q_flow) * (nu[0, 2] * r1 + nu[1, 2] * r2)
    resid_D = C_D0 - C_D + (V / Q_flow) * (nu[0, 3] * r1 + nu[1, 3] * r2)
    
    # Energy Balance (Robust Form)
    h_A, h_B, h_C, h_D = H(T)
    delta_h = (
        (C_A0 * H(T0)[0] + C_B0 * H(T0)[1] + C_C0 * H(T0)[2] + C_D0 * H(T0)[3]) -
        (C_A * h_A + C_B * h_B + C_C * h_C + C_D * h_D)
    )
    
    Q_rxn = (delta_H_rxn1 * r1 + delta_H_rxn2 * r2) * V  # kJ/hr
    
    # Preventing divide by zero and maintaining stability
    slope_sum = np.sum(hl_data[:, 1] * [C_A, C_B, C_C, C_D])
    if abs(slope_sum) < 1e-6:
        slope_sum = 1e-6  # Prevent division by zero
    
    resid_T = ((delta_h + Q_rxn) / Q_flow - np.sum(hl_data[:, 0] * [C_A, C_B, C_C, C_D])) / slope_sum
    return [resid_A, resid_B, resid_C, resid_D, resid_T]

# Smart initial guess
guess = [C_A0 , C_B0 , C_C0 , C_D0 , T0]

# Setting bounds (non-negative concentrations and a reasonable temperature range)
bounds = (
    [0, 0, 0, 0, 200],  # Lower bounds (non-negative conc, T > 200 K)
    [C_A0 * 10, C_B0 * 10, 10,  10, 524]  # Upper bounds
)

# Solving using least_squares with bounds
result = least_squares(
    equations, 
    guess, 
    bounds=bounds, 
    method='trf', 
    xtol=1e-8,
    ftol=1e-8,
    gtol=1e-8,
    max_nfev=5000
)

# Displaying results
if result.success:
    C_A_sol, C_B_sol, C_C_sol, C_D_sol, T_sol = result.x
    print("Adiabatic CSTR Results (Bounded Solver - least_squares):")
    print(f"C_A: {C_A_sol:.5f} kmol/m³")
    print(f"C_B: {C_B_sol:.5f} kmol/m³")
    print(f"C_C: {C_C_sol:.5f} kmol/m³")
    print(f"C_D: {C_D_sol:.5f} kmol/m³")
    print(f"Temperature: {T_sol:.2f} K")
else:
    print("Solution did not converge:", result.message)


Adiabatic CSTR Results (Bounded Solver - least_squares):
C_A: 0.00000 kmol/m³
C_B: 3.11454 kmol/m³
C_C: 0.00000 kmol/m³
C_D: 0.00000 kmol/m³
Temperature: 524.00 K


In [ ]:
import numpy as np
from scipy.optimize import root

# Reactor parameters (default values)
V = 1.0   # Volume of the CSTR (L)
F = 1.0   # Volumetric flow rate (L/min)
T_in = 300.0 # Inlet temperature in Kelvin (default)

# Enthalpy and heat capacity (example values, can be modified)
dH1 = -50000.0  # Enthalpy change for A + B -> C (J/mol)
dH2 = -75000.0  # Enthalpy change for A + C -> D (J/mol)
Cp = 100.0      # Heat capacity (J/mol·K) - assumed same for all species

# Inlet molar fractions (default values)
YA0 = 0.5
YB0 = 0.5
YC0 = 0.0
YD0 = 0.0

# Function to define the system of equations
def cstr_equations(vars, V, F, T_in, dH1, dH2, Cp, YA0, YB0, YC0, YD0):
    cA, cB, cC, cD, T = vars

    # Updated rate constants with temperature dependence
    k1 = 13706.91 * np.exp(-8220 / T)
    k2 = 96341.59 * np.exp(-8700 / T)

    # Material balance equations
    eq1 = F * (YA0 - cA) - V * (k1 * cA * cB + k2 * cA * cC)
    eq2 = F * (YB0 - cB) - V * (k1 * cA * cB)
    eq3 = F * (YC0 - cC) + V * (k1 * cA * cB - k2 * cA * cC)
    eq4 = F * (YD0 - cD) + V * (k2 * cA * cC)

    # Energy balance
    eq5 = F * Cp * (T_in - T) + V * (dH1 * k1 * cA * cB + dH2 * k2 * cA * cC)

    return [eq1, eq2, eq3, eq4, eq5]

# Solver function
def solve_cstr(V=1.0, F=1.0, T_in=300.0, dH1=-50000.0, dH2=-75000.0, Cp=100.0, YA0=0.5, YB0=0.5, YC0=0.0, YD0=0.0):
    initial_guess = [0.5, 0.5, 0.5, 0.5, 300.0]  # Initial guess for concentrations and temperature

    solution = root(cstr_equations, initial_guess, args=(V, F, T_in, dH1, dH2, Cp, YA0, YB0, YC0, YD0))

    if solution.success:
        cA, cB, cC, cD, T = solution.x
        print(f"Final concentrations:\n A: {cA:.4f} M\n B: {cB:.4f} M\n C: {cC:.4f} M\n D: {cD:.4f} M\nFinal Temperature: {T:.2f} K")
    else:
        print("Solution not found.")

# Example usage (can be modified)
if __name__ == "__main__":
    solve_cstr(V=1.0, F=1.0, T_in=300.0, dH1=-50000.0, dH2=-75000.0, Cp=100.0, YA0=0.5, YB0=0.5, YC0=0.0, YD0=0.0)

In [ ]:
def solve(self, inlet_stream):
        """Solve adiabatic CSTR for outlet conditions"""
        # Inlet conditions
        F_in = inlet_stream.F
        z_in = inlet_stream.z
        T_in = inlet_stream.T
        P_in = inlet_stream.P
        
        # Initial guesses [F_EO, F_H2O, F_EG, F_DEG, T_out]
        guess = [
            F_in * z_in[0] * 0.8,   # 20% EO conversion guess
            F_in * z_in[1] * 0.99,  # Small H2O change
            F_in * z_in[2] + 5,     # Some EG produced
            F_in * z_in[3] + 1,     # Small DEG produced
            T_in + 10               # Slightly higher T
        ]
        
        def equations(vars):
            F_EO, F_H2O, F_EG, F_DEG, T_out = vars
            
            # Concentrations (kmol/m³)
            # Assuming constant density (approx. v = F_in/ρ where ρ ≈ total kmol/m³)
            v = F_in / sum(z_in)  # m³/h
            C = np.array([F_EO, F_H2O, F_EG, F_DEG]) / v
            
            # Reaction rates
            k1, k2 = self.reaction_rates(T_out)
            r1 = k1 * C[0] * C[1]  # EO + H2O → EG
            r2 = k2 * C[2] * C[0]  # EG + EO → DEG
            
            # Material balances
            resid_EO = (F_in * z_in[0] - F_EO) - (r1 + r2) * self.volume
            resid_H2O = (F_in * z_in[1] - F_H2O) - r1 * self.volume
            resid_EG = (F_in * z_in[2] - F_EG) - (-r1 + r2) * self.volume
            resid_DEG = (F_in * z_in[3] - F_DEG) - (-r2) * self.volume
            
            # Energy balance (H_in = H_out)
            h_in = inlet_stream.H()
            outlet_stream = Stream('temp', T_out, P_in, sum([F_EO, F_H2O, F_EG, F_DEG]), 
                                 [F_EO, F_H2O, F_EG, F_DEG]/sum([F_EO, F_H2O, F_EG, F_DEG]))
            h_out = outlet_stream.H()
            resid_energy = h_in - h_out
            
            return [resid_EO, resid_H2O, resid_EG, resid_DEG, resid_energy]
        
        # Solve nonlinear equations
        solution = fsolve(equations, guess)
        F_EO, F_H2O, F_EG, F_DEG, T_out = solution
        
        # Create outlet stream
        F_out = sum([F_EO, F_H2O, F_EG, F_DEG])
        z_out = [F_EO/F_out, F_H2O/F_out, F_EG/F_out, F_DEG/F_out]
        outlet_stream = Stream(
            f"{inlet_stream.name}_outlet", 
            T_out, 
            P_in, 
            F_out, 
            z_out
        )
        
        # Calculate conversion
        X_EO = (F_in * z_in[0] - F_EO) / (F_in * z_in[0])
        
        return outlet_stream, X_EO

In [31]:
import numpy as np
from scipy.optimize import fsolve

class Stream:
    def __init__(self, name, T, P, F, z):
        self.name = name
        self.T = T  # K
        self.P = P  # atm
        self.F = F  # kmol/h
        self.z = np.array(z)  # [EO, H2O, EG, DEG]
        
    def liquid_molar_density(self):
        """Calculate component molar densities (kmol/m³) using Eq.39"""
        rho_params = np.array([
            [1.655, 0.126300, -0.0002286],    # EO
            [75.22, -0.065050, 0.000000],      # H2O
            [23.66, -0.018040, 0.000000],      # EG
            [13.59, -0.009986, 0.000000]       # DEG
        ])
        return rho_params[:,0] + rho_params[:,1]*self.T + rho_params[:,2]*self.T**2
        
    def mixture_molar_density(self):
        """Calculate mixture molar density (kmol/m³)"""
        rho_pure = self.liquid_molar_density()
        # Simple ideal mixing rule (can be modified if needed)
        return np.sum(self.z * rho_pure)
        
    def H(self):
        """Calculate liquid-phase enthalpy (kJ/h)"""
        hl_data = np.array([
            [-143.8, 0.2037],   # EO
            [-320.0, 0.1068],    # H2O
            [-519.0, 0.1924],    # EG
            [-723.0, 0.3043]     # DEG
        ])
        h = hl_data[:,0] + hl_data[:,1] * self.T
        return np.dot(h, self.z) * self.F

class CSTReactor:
    def __init__(self, volume):
        self.volume = volume  # m³
        
    def reaction_rates(self, T):
        """Return k1 and k2 at temperature T (K) in m³/(kmol·h)"""
        k1 = 13706.91 * np.exp(-8220 / T)  # m³/(kmol·h)
        k2 = 96341.59 * np.exp(-8700 / T)   # m³/(kmol·h)
        return k1, k2
        
    def solve(self, inlet_stream):
        """Solve adiabatic CSTR for outlet conditions"""
        # Inlet conditions
        F_in = inlet_stream.F
        z_in = inlet_stream.z
        T_in = inlet_stream.T
        P_in = inlet_stream.P
        
        # Initial guesses
        guess = [
            F_in * z_in[0] ,   # Assume 40% EO conversion
            F_in * z_in[1] ,   # H2O consumption
            F_in * z_in[2] ,     # EG production
            F_in * z_in[3] ,      # DEG production
            T_in +10                # Temperature rise
        ]
        
        def equations(vars):
            F_EO, F_H2O, F_EG, F_DEG, T_out = vars
            
            # Create temporary stream for density calculation
            temp_stream = Stream('temp', T_out, P_in, 
                               sum([F_EO, F_H2O, F_EG, F_DEG]),
                               [F_EO, F_H2O, F_EG, F_DEG])
            
            # Calculate volumetric flow using exact density
            rho_mix = temp_stream.mixture_molar_density()
            v = (F_EO + F_H2O + F_EG + F_DEG) / rho_mix  # m³/h
            
            # Concentrations (kmol/m³)
            C = np.array([F_EO, F_H2O, F_EG, F_DEG]) / v
            
            # Reaction rates
            k1, k2 = self.reaction_rates(T_out)
            r1 = k1 * C[0] * C[1]  # EO + H2O → EG
            r2 = k2 * C[2] * C[0]  # EG + EO → DEG
            
            # Material balances
            resid_EO = (F_in * z_in[0] - F_EO) - (r1 + r2) * self.volume
            resid_H2O = (F_in * z_in[1] - F_H2O) - r1 * self.volume
            resid_EG = (F_in * z_in[2] - F_EG) - (-r1 + r2) * self.volume
            resid_DEG = (F_in * z_in[3] - F_DEG) - (-r2) * self.volume
            
            # Energy balance
            h_in = inlet_stream.H()
            h_out = temp_stream.H()
            resid_energy = h_in - h_out
            
            return [resid_EO, resid_H2O, resid_EG, resid_DEG, resid_energy]
        
        solution = fsolve(equations, guess, xtol=1e-6)
        F_EO, F_H2O, F_EG, F_DEG, T_out = solution
        
        # Create final outlet stream
        F_out = sum([F_EO, F_H2O, F_EG, F_DEG])
        z_out = [F_EO/F_out, F_H2O/F_out, F_EG/F_out, F_DEG/F_out]
        outlet_stream = Stream(f"{inlet_stream.name}_outlet", T_out, P_in, F_out, z_out)
        X_EO = (F_in * z_in[0] - F_EO) / (F_in * z_in[0])
        
        return outlet_stream, X_EO

# Test case
inlet = Stream("feed", 355, 2.4, 582, [0.048, 0.951, 0.0001, 0])
cstr = CSTReactor(3.75)
outlet, conversion = cstr.solve(inlet)

# Results
print(f"Outlet Temperature: {outlet.T:.1f} K (ΔT = {outlet.T - inlet.T:.1f} K)")
print(f"EO Conversion: {conversion*100:.1f}%")
print("\nMole Fractions:")
print(f"  EO: {outlet.z[0]:.5f} (inlet: {inlet.z[0]:.5f})")
print(f"  H2O: {outlet.z[1]:.5f} (inlet: {inlet.z[1]:.5f})")
print(f"  EG: {outlet.z[2]:.5f} (inlet: {inlet.z[2]:.5f})")
print(f"  DEG: {outlet.z[3]:.5f} (inlet: {inlet.z[3]:.5f})")

# Density verification
print("\nDensity Verification:")
print(f"Inlet density: {inlet.mixture_molar_density():.2f} kmol/m³")
print(f"Outlet density: {outlet.mixture_molar_density():.2f} kmol/m³")

Outlet Temperature: 11.0 K (ΔT = -344.0 K)
EO Conversion: 2078.8%

Mole Fractions:
  EO: -745.38442 (inlet: 0.04800)
  H2O: -39.61182 (inlet: 0.95100)
  EG: 788.85970 (inlet: 0.00010)
  DEG: -2.86345 (inlet: 0.00000)

Density Verification:
Inlet density: 50.42 kmol/m³
Outlet density: 13265.28 kmol/m³


C:\Users\ATK\AppData\Local\Temp\ipykernel_3164\978396940.py:45: RuntimeWarning: overflow encountered in exp
  k1 = 13706.91 * np.exp(-8220 / T)  # m³/(kmol·h)
C:\Users\ATK\AppData\Local\Temp\ipykernel_3164\978396940.py:46: RuntimeWarning: overflow encountered in exp
  k2 = 96341.59 * np.exp(-8700 / T)   # m³/(kmol·h)
C:\Users\ATK\AppData\Local\Temp\ipykernel_3164\978396940.py:87: RuntimeWarning: invalid value encountered in scalar add
  resid_EO = (F_in * z_in[0] - F_EO) - (r1 + r2) * self.volume
C:\Users\ATK\AppData\Local\Temp\ipykernel_3164\978396940.py:99: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  solution = fsolve(equations, guess, xtol=1e-6)


In [ ]:
import numpy as np
from scipy.optimize import fsolve

class Stream:
    def __init__(self, name, T, P, F, z):
        self.name = name
        self.T = T  # K
        self.P = P  # atm
        self.F = F  # kmol/h
        self.z = np.array(z)  # [EO, H2O, EG, DEG]
        
    def H(self):
        """Calculate liquid-phase enthalpy (kJ/h)"""
        # Enthalpy coefficients [a, b] where h = a + b*T (kJ/kmol)
        hl_data = np.array([
            [-143.8, 0.2037],   # EO
            [-320.0, 0.1068],    # H2O
            [-519.0, 0.1924],    # EG
            [-723.0, 0.3043]     # DEG
        ])
        h = hl_data[:,0] + hl_data[:,1] * self.T
        return np.dot(h, self.z) * self.F

class CSTReactor:
    def __init__(self, volume):
        self.volume = volume  # m³
        
    def reaction_rates(self, T):
        """Return k1 and k2 at temperature T (K) in m³/(kmol·h)"""
        k1 = 13706.91 * np.exp(-8220 / T)  # m³/(kmol·h)
        k2 = 96341.59 * np.exp(-8700 / T)   # m³/(kmol·h)
        return k1, k2
        
    def solve(self, inlet_stream):
        """Solve adiabatic CSTR for outlet conditions"""
        # Inlet conditions
        F_in = inlet_stream.F
        z_in = inlet_stream.z
        T_in = inlet_stream.T
        P_in = inlet_stream.P
        
        # Initial guesses [F_EO, F_H2O, F_EG, F_DEG, T_out]
        # More aggressive guesses to force reaction
        guess = [
            F_in * z_in[0] ,   # 50% EO conversion guess
            F_in * z_in[1] ,  # H2O consumption
            F_in * z_in[2] ,    # EG production
            F_in * z_in[3] ,     # DEG production
            T_in                # Temperature rise
        ]
        
        def equations(vars):
            F_EO, F_H2O, F_EG, F_DEG, T_out = vars
            
            # Total outlet flow for concentration calculation
            F_total = F_EO + F_H2O + F_EG + F_DEG
            
            # Concentrations (kmol/m³)
            # Assuming liquid density ρ ≈ 55 kmol/m³ (typical for water mixtures)
            v = F_total / 55  # m³/h volumetric flow rate
            C = np.array([F_EO, F_H2O, F_EG, F_DEG]) / v
            
            # Reaction rates (kmol/m³·h)
            k1, k2 = self.reaction_rates(T_out)
            r1 = k1 * C[0] * C[1]  # EO + H2O → EG
            r2 = k2 * C[2] * C[0]  # EG + EO → DEG
            
            # Material balances (kmol/h)
            resid_EO = (F_in * z_in[0] - F_EO) - (r1 + r2) * self.volume
            resid_H2O = (F_in * z_in[1] - F_H2O) - r1 * self.volume
            resid_EG = (F_in * z_in[2] - F_EG) - (-r1 + r2) * self.volume
            resid_DEG = (F_in * z_in[3] - F_DEG) - (-r2) * self.volume
            
            # Energy balance (H_in = H_out)
            h_in = inlet_stream.H()
            outlet_stream = Stream('temp', T_out, P_in, F_total, 
                                 [F_EO, F_H2O, F_EG, F_DEG]/F_total)
            h_out = outlet_stream.H()
            resid_energy = h_in - h_out
            
            return [resid_EO, resid_H2O, resid_EG, resid_DEG, resid_energy]
        
        # Solve nonlinear equations with better tolerance
        solution = fsolve(equations, guess, xtol=1e-6)
        F_EO, F_H2O, F_EG, F_DEG, T_out = solution
        
        # Create outlet stream
        F_out = F_EO + F_H2O + F_EG + F_DEG
        z_out = [F_EO/F_out, F_H2O/F_out, F_EG/F_out, F_DEG/F_out]
        outlet_stream = Stream(
            f"{inlet_stream.name}_outlet", 
            T_out, 
            P_in, 
            F_out, 
            z_out
        )
        
        # Calculate conversion
        X_EO = (F_in * z_in[0] - F_EO) / (F_in * z_in[0])
        
        return outlet_stream, X_EO

# Define inlet stream (from Heater1.outlet in your original code)
inlet_stream = Stream(
    name="CSTR_inlet",
    T=355,      # K
    P=2.4,      # atm
    F=582,      # kmol/h
    z=[0.048, 0.951, 0.0001, 0]  # [EO, H2O, EG, DEG]
)

# Create and solve CSTR
cstr = CSTReactor(volume=3.75)  # m³
outlet_stream, X_EO = cstr.solve(inlet_stream)

# Print results
print("\nAdiabatic CSTR Results (V = 3.75 m³)")
print("----------------------------------")
print(f"Inlet Temperature: {inlet_stream.T} K")
print(f"Outlet Temperature: {outlet_stream.T:.1f} K")
print(f"Pressure: {outlet_stream.P} atm")
print(f"Total Molar Flow Out: {outlet_stream.F:.1f} kmol/h")
print("\nCompositions (mol frac):")
print(f"EO: {outlet_stream.z[0]:.4f} (was {inlet_stream.z[0]:.4f})")
print(f"H2O: {outlet_stream.z[1]:.4f} (was {inlet_stream.z[1]:.4f})")
print(f"EG: {outlet_stream.z[2]:.4f} (was {inlet_stream.z[2]:.4f})")
print(f"DEG: {outlet_stream.z[3]:.4f} (was {inlet_stream.z[3]:.4f})")
print(f"\nEO Conversion: {X_EO*100:.1f}%")


Adiabatic CSTR Results (V = 3.75 m³)
----------------------------------
Inlet Temperature: 355 K
Outlet Temperature: 355.0 K
Pressure: 2.4 atm
Total Molar Flow Out: 581.5 kmol/h

Compositions (mol frac):
EO: 0.0480 (was 0.0480)
H2O: 0.9519 (was 0.9510)
EG: 0.0001 (was 0.0001)
DEG: 0.0000 (was 0.0000)

EO Conversion: 0.0%


In [ ]:
class CSTReactor(ProcessUnits):
    def __init__(self, name, inlet_stream, volume=0):
        super().__init__(name)
        self.inlet_stream = inlet_stream
        self.volume = volume
        self.T = inlet_stream.T  # Initial temperature from the inlet stream
        self.P = inlet_stream.P  # Pressure from the inlet stream
        self.F = inlet_stream.F  # Molar flow rate from the inlet stream
        self.z = inlet_stream.z  # Molar fractions from the inlet stream
        self.Hin = inlet_stream.H()  # Enthalpy from the inlet stream
        self.Hout = 0  # Initialize outlet enthalpy
        self.Q = 0  # Initialize heat duty
        
        # Initial concentrations (assuming they follow inlet composition)
        self.C_A = self.F * self.z[0]  # Concentration of A
        self.C_B = self.F * self.z[1]  # Concentration of B
        self.C_C = self.F * self.z[2]  # Concentration of C
        self.C_D = self.F * self.z[3]  # Concentration of D

    def reaction_rates(self, T):
        """Calculate reaction rates r1 and r2 at given temperature T."""
        k1 = 13706.91 * np.exp(-8220 / T)
        k2 = 96341.59 * np.exp(-8700 / T)
        return k1, k2
    
    def H(self,T):
        hl_data = np.array([
            [-143.8, 0.2037],
            [-320, 0.1068],
            [-519, 0.1924],
            [-723, 0.3043],
            ])
        hls = hl_data[:,0] + hl_data[:,1]*T
        return 
    
    def mass_balance(self, C_A, C_B, C_C, C_D, V, T):
        """Calculate the rate of change of concentrations for mass balance."""
        k1, k2 = self.reaction_rates(T)
        
        # Mass balance equations for species A, B, C, and D
        dC_A = -k1 * C_A * C_B * V  # A is consumed by r1
        dC_B = -k1 * C_A * C_B * V  # B is consumed by r1
        dC_C = (k1 * C_A * C_B - k2 * C_A * C_C) * V  # C is produced by r1, consumed by r2
        dC_D = k2 * C_A * C_C * V  # D is produced by r2
        
        return np.array([dC_A, dC_B, dC_C, dC_D])

    def solve_steady_state(self, max_iterations=1000, tolerance=1e-5):
        """Solve for steady-state composition using iterative method."""
        C_A, C_B, C_C, C_D = self.C_A, self.C_B, self.C_C, self.C_D
        V = self.volume
        T = self.T  # Temperature from inlet stream
        
        for iteration in range(max_iterations):
            # Mass balance for each species
            dC = self.mass_balance(C_A, C_B, C_C, C_D, V, T)
            
            # Update concentrations
            C_A_new = C_A + dC[0]
            C_B_new = C_B + dC[1]
            C_C_new = C_C + dC[2]
            C_D_new = C_D + dC[3]
            
            # Update Temperature based on the new concentrations
            

            # Check for convergence
            if np.all(np.abs(dC) < tolerance):
                print(f"Steady-state reached after {iteration+1} iterations")
                break
            
            # Update the concentrations for the next iteration
            C_A, C_B, C_C, C_D = C_A_new, C_B_new, C_C_new, C_D_new
        
        # Update final values
        self.C_A, self.C_B, self.C_C, self.C_D = C_A, C_B, C_C, C_D
        return C_A, C_B, C_C, C_D

# Example Usage
Feed1 = Stream('Feed1', 298, 2.4, 26.32, z=[0, 1, 0, 0])
Feed2 = Stream('Feed2', 298, 2.4, 27.62, z=[1, 0, 0, 0])
D1 = Stream('D1', 360.5, 2.4, 528.018, z=[0.0088, 0.9911, 0.0001, 0])
Mixer1 = Mixer('Mixer1', Feed1, Feed2, D1)

Heater1 = Heater('Heater1', Mixer1.outlet, 355)

# Set reactor with volume 3.75 m3
reactor = CSTReactor('Reactor1', Heater1.outlet, volume=)

# Solve for steady-state composition
C_A, C_B, C_C, C_D = reactor.solve_steady_state()

# Print final composition
print(f"\nFinal Composition at Steady State:")
print(f"C_A = {C_A:.2f} kmol/h")
print(f"C_B = {C_B:.2f} kmol/h")
print(f"C_C = {C_C:.2f} kmol/h")
print(f"C_D = {C_D:.2f} kmol/h")



Final Composition at Steady State:
C_A = 1.34 kmol/h
C_B = 518.71 kmol/h
C_C = 29.39 kmol/h
C_D = 1.59 kmol/h
